In [4]:
import numpy as np
import pandas as pd
import quandl
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [99]:
#Getting the stock data
df = quandl.get("WIKI/AMZN")
print (df.columns.tolist())
['Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend', 'Split Ratio',
 'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']
df.head()

['Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend', 'Split Ratio', 'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']


,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
1997-05-16,22.38,23.75,20.50,20.75,1225000.0,0.0,1.0,1.865000,1.979167,1.708333,1.729167,14700000.0
1997-05-19,20.50,21.25,19.50,20.50,508900.0,0.0,1.0,1.708333,1.770833,1.625000,1.708333,6106800.0
1997-05-20,20.75,21.00,19.63,19.63,455600.0,0.0,1.0,1.729167,1.750000,1.635833,1.635833,5467200.0
1997-05-21,19.25,19.75,16.50,17.13,1571100.0,0.0,1.0,1.604167,1.645833,1.375000,1.427500,18853200.0
1997-05-22,17.25,17.38,15.75,16.75,981400.0,0.0,1.0,1.437500,1.448333,1.312500,1.395833,11776800.0


In [100]:
df = df[['Adj. Close']]
#Take a look at the new data
print(df.head())

            Adj. Close
Date                  
1997-05-16    1.729167
1997-05-19    1.708333
1997-05-20    1.635833
1997-05-21    1.427500
1997-05-22    1.395833


In [101]:
forecast_out = 30 #'n=30' days
#Create another column (the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
#print the new data set
print(df.tail())

            Adj. Close  Prediction
Date                              
2018-03-21     1581.86         NaN
2018-03-22     1544.10         NaN
2018-03-23     1495.56         NaN
2018-03-26     1555.86         NaN
2018-03-27     1497.05         NaN


In [102]:
#Create the independent array based on which we are going to make the prediction
X = np.array(df.drop(['Prediction'],1))

X= X[: -forecast]

print(X)


[[   1.72916667]
 [   1.70833333]
 [   1.63583333]
 ...
 [1350.47      ]
 [1338.99      ]
 [1386.23      ]]


In [103]:
#THe variable we want to predict
Y = np.array(df['Prediction'])
# Get all of the y values except the last 'n' rows
Y = Y[:-forecast_out]
print(Y)



[1.54166667e+00 1.51583333e+00 1.58833333e+00 ... 1.49556000e+03
 1.55586000e+03 1.49705000e+03]


In [108]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4)

In [109]:
#Create and Train the support vector machine
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.4)
svr_rbf.fit(X_train, Y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.4,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [110]:
svm_confidence = svr_rbf.score(X_test, Y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.8901236368398693


In [111]:
#Create and Train the Linear Regression model
lm = LinearRegression()
lm.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [113]:
lm_confidence = lm.score(X_test, Y_test)
print("Lm confidence: ", lm_confidence)

Lm confidence:  0.9889742418675136


In [114]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[1414.51]
 [1451.05]
 [1461.76]
 [1448.69]
 [1468.35]
 [1482.92]
 [1484.76]
 [1500.  ]
 [1521.95]
 [1511.98]
 [1512.45]
 [1493.45]
 [1500.25]
 [1523.61]
 [1537.64]
 [1545.  ]
 [1551.86]
 [1578.89]
 [1598.39]
 [1588.18]
 [1591.  ]
 [1582.32]
 [1571.68]
 [1544.93]
 [1586.51]
 [1581.86]
 [1544.1 ]
 [1495.56]
 [1555.86]
 [1497.05]]


In [119]:
#The linear regression model predictions for the next 'n' days
print(lm.predict(x_forecast))

[1501.27768767 1540.14377898 1551.53556436 1537.63354429 1558.54507563
 1574.04258384 1575.99971597 1592.20987556 1615.55718547 1604.95250758
 1605.4524272  1585.24291065 1592.47579026 1617.32285902 1632.24599151
 1640.07452003 1647.37121916 1676.1219156  1696.86326153 1686.00330553
 1689.00282325 1679.77026517 1668.4529359  1640.00006391 1684.2269954
 1679.28098213 1639.11722714 1587.48723064 1651.62585422 1589.0720822 ]


In [120]:
#The support vector regressor model predictions for the next 'n' days
print(svr_rbf.predict(x_forecast))

[ 702.87568789 1446.0624394   580.17469134  610.23228347  580.17469134
  580.17469134  580.17469134  580.17469134  580.17469134  580.17469134
  580.17469134  580.17469134  580.17469134  580.17469134  580.17469134
  580.17469134  580.17469134  580.17469134  580.17469134  580.17469134
  580.17469134  580.17469134  580.17469134  580.17469134  580.17469134
  580.17469134  580.17469134  580.17469134  580.17469134  580.17469134]
